# Course Work - Facial Expression Recoginnision

Module: Computer Vision - INM460

By: Asha Guruvayurappan

## **Support Vector Machine with HOG**

Implementing a series of image classification models using dataset from https://forms.gle/P7jK9TThhUjvUuvi9



### Google Colab Setup
Before starting, we need to run a few commands to set up our environment on Google Colab. Also, if running this notebook on a local machine you can skip this section. 

The following cell is to mount the Google Drive

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os

# TODO: Fill in the Google Drive path where you uploaded the lab materials
# Example: GOOGLE_DRIVE_PATH_AFTER_MYDRIVE = 'Colab Notebooks/Lab materials 01-20210104'

GOOGLE_DRIVE_PATH_AFTER_MYDRIVE = 'ColabNotebooks/ComputerVision/Course Work/CW_Folder_PG' 
GOOGLE_DRIVE_PATH = os.path.join('drive', 'My Drive', GOOGLE_DRIVE_PATH_AFTER_MYDRIVE)
print(os.listdir(GOOGLE_DRIVE_PATH))

['.DS_Store', 'Video', 'CW_Dataset', 'Code', 'Models', 'test_functions.ipynb']


### Data Loading

The dataset taken from the link is a ziped folder placed inside 'CW_Dataset'. The following lines of code will unzip the folder and create a copy named 'CW_Dataset-2'


In [3]:
zip_path = os.path.join(GOOGLE_DRIVE_PATH, 'CW_Dataset', 'CW_Dataset-2.zip')

# Copy it to Colab
!cp '{zip_path}' .

# Unzip it
!yes|unzip -q CW_Dataset-2.zip

# Delete zipped version from Colab (not from Drive)
# !rm CW_Dataset-2.zip

In [23]:
from sklearn import svm, metrics
from sklearn.utils import shuffle
import matplotlib.pyplot as plt
import numpy as np
import cv2
from sklearn.model_selection import train_test_split
from skimage import img_as_ubyte, io, color
# from sklearn.cluster import MiniBatchKMeans
from collections import Counter
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
import joblib
from skimage.feature import hog
from skimage import data, exposure
from sklearn.model_selection import cross_val_score

from sklearn.neural_network import MLPClassifier

import numpy as np
import joblib
from joblib import dump, load

%matplotlib inline


### Data Loading

The dataset taken from the link is a ziped folder placed inside 'CW_Dataset'. The following lines of code will unzip the folder and create a copy named 'CW_Dataset-2'


In [5]:
def import_training_data(path, folder, lable_list=None):
  """Load training images and lables from folders"""

  images =[]
  labels = []
  file_names = [file for file in sorted(os.listdir(os.path.join(path,folder))) if file.endswith('.jpg')]
  print(file_names)
  for file in file_names:
    images.append(io.imread(os.path.join(path, folder, file)))
  
  file_names = [file.replace('_aligned','') for file in file_names]
  
  return images, file_names
  


path = '/content/CW_Dataset-2'
images, file_names = import_training_data(path,'train')
print(file_names)

['train_00001_aligned.jpg', 'train_00002_aligned.jpg', 'train_00003_aligned.jpg', 'train_00004_aligned.jpg', 'train_00005_aligned.jpg', 'train_00006_aligned.jpg', 'train_00007_aligned.jpg', 'train_00008_aligned.jpg', 'train_00009_aligned.jpg', 'train_00010_aligned.jpg', 'train_00011_aligned.jpg', 'train_00012_aligned.jpg', 'train_00013_aligned.jpg', 'train_00014_aligned.jpg', 'train_00015_aligned.jpg', 'train_00016_aligned.jpg', 'train_00017_aligned.jpg', 'train_00018_aligned.jpg', 'train_00019_aligned.jpg', 'train_00020_aligned.jpg', 'train_00021_aligned.jpg', 'train_00022_aligned.jpg', 'train_00023_aligned.jpg', 'train_00024_aligned.jpg', 'train_00025_aligned.jpg', 'train_00026_aligned.jpg', 'train_00027_aligned.jpg', 'train_00028_aligned.jpg', 'train_00029_aligned.jpg', 'train_00030_aligned.jpg', 'train_00031_aligned.jpg', 'train_00032_aligned.jpg', 'train_00033_aligned.jpg', 'train_00034_aligned.jpg', 'train_00035_aligned.jpg', 'train_00036_aligned.jpg', 'train_00037_aligned.jpg', 

In [6]:
lables_path = os.path.join(path, 'labels')
file_labels= sorted(open(os.path.join(lables_path, 'list_label_train.txt'), "r").readlines())
# content_list = labels.readlines()
file_labels = [line.rstrip('\n') for line in file_labels]
print(file_labels)

labels = [label[-1] for label in file_labels]
print(labels)

['train_00001.jpg 5', 'train_00002.jpg 5', 'train_00003.jpg 4', 'train_00004.jpg 4', 'train_00005.jpg 5', 'train_00006.jpg 1', 'train_00007.jpg 5', 'train_00008.jpg 4', 'train_00009.jpg 4', 'train_00010.jpg 1', 'train_00011.jpg 4', 'train_00012.jpg 1', 'train_00013.jpg 1', 'train_00014.jpg 4', 'train_00015.jpg 5', 'train_00016.jpg 4', 'train_00017.jpg 4', 'train_00018.jpg 4', 'train_00019.jpg 4', 'train_00020.jpg 4', 'train_00021.jpg 4', 'train_00022.jpg 4', 'train_00023.jpg 6', 'train_00024.jpg 5', 'train_00025.jpg 5', 'train_00026.jpg 5', 'train_00027.jpg 2', 'train_00028.jpg 5', 'train_00029.jpg 4', 'train_00030.jpg 5', 'train_00031.jpg 3', 'train_00032.jpg 1', 'train_00033.jpg 5', 'train_00034.jpg 1', 'train_00035.jpg 4', 'train_00036.jpg 4', 'train_00037.jpg 4', 'train_00038.jpg 5', 'train_00039.jpg 5', 'train_00040.jpg 5', 'train_00041.jpg 5', 'train_00042.jpg 5', 'train_00043.jpg 1', 'train_00044.jpg 1', 'train_00045.jpg 4', 'train_00046.jpg 4', 'train_00047.jpg 6', 'train_00048

### **Train - Validation split**

10% of training data is split as validation set


In [7]:
X_train, X_val, y_train, y_val = train_test_split(images, labels, test_size = 0.1)

### **HOG**

To extract features using HOG the images are first converted into Grey Scale images. and the Hog features are appended into a `HOG_des` list

Performing HOG on training set

*Adapted from :  https://scikit-image.org/docs/dev/auto_examples/features_detection/plot_hog.html*



In [8]:
HOG_des = []
HOG_image = []
grey_image = X_train.copy()

for i in range(len(X_train)):
    # Identify keypoints and extract descriptors with HOG
    grey_image[i] = color.rgb2gray(X_train[i])

for image in grey_image:
  des, h_image = hog(image, orientations=8, pixels_per_cell=(16, 16), cells_per_block=(1, 1), visualize=True,)
  HOG_des.append(des)
  HOG_image.append(h_image)

In [9]:
hog_features = np.array(HOG_des)

### **Support Vector Machine**



*   Multiple models are created with different parameters to check the best set of parameters.
*   Grid search is used to tune hyperparameters. 
*   Accuracy on validation set is used as a metric to evaluate model performance




**SVM Model -1** 

SVM model is trained with cross validation with CV partition = 5, to avoid over fitting. 

*Ref : https://scikit-learn.org/stable/modules/cross_validation.html*


In [14]:
svm_classifierCross = svm.SVC(kernel='rbf')

scores = cross_val_score(svm_classifierCross, hog_features, y_train, cv=5, verbose=True)


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:  1.7min finished


In [15]:
scores

array([0.63241286, 0.62607515, 0.61385242, 0.60960145, 0.61096014])

In [14]:
svm_classifierCross.fit(hog_features, y_train)

SVC()

**Validation set**


Performing HOG on validation set to identify the accuracy

In [10]:
HOG_des_val = []
HOG_image_val = []
grey_image_val = X_val.copy()

for i in range(len(X_val)):
    # Identify keypoints and extract descriptors with SIFT
    grey_image_val[i] = color.rgb2gray(X_val[i])

for image in grey_image_val:
  des, h_image = hog(image, orientations=8, pixels_per_cell=(16, 16), cells_per_block=(1, 1), visualize=True,)
  HOG_des_val.append(des)
  HOG_image_val.append(h_image)


hog_features_val = np.array(HOG_des_val)

In [15]:
ypreds_val = svm_classifierCross.predict(hog_features_val)
print(classification_report(ypreds_val, y_val))

              precision    recall  f1-score   support

           1       0.54      0.65      0.59       116
           2       0.14      0.80      0.24         5
           3       0.04      0.33      0.07         9
           4       0.88      0.71      0.79       570
           5       0.43      0.59      0.49       157
           6       0.36      0.56      0.44        48
           7       0.70      0.51      0.59       323

    accuracy                           0.63      1228
   macro avg       0.44      0.59      0.46      1228
weighted avg       0.71      0.63      0.66      1228



**GRID SEARCH**


In [23]:
param_grid = {'C': [0.1, 1, 10, 100],
              'gamma': [1, 0.1, 0.01],
              'kernel': ['linear', 'rbf']}
 
grid_SVM_hog = GridSearchCV(svm.SVC(), param_grid, refit = True, verbose = 3)

grid_SVM_hog_model = grid_SVM_hog.fit(hog_features, y_train)

print(grid_SVM_hog_model.best_params_)

Fitting 5 folds for each of 24 candidates, totalling 120 fits
[CV 1/5] END .....C=0.1, gamma=1, kernel=linear;, score=0.593 total time=  15.5s
[CV 2/5] END .....C=0.1, gamma=1, kernel=linear;, score=0.600 total time=  16.5s
[CV 3/5] END .....C=0.1, gamma=1, kernel=linear;, score=0.581 total time=  12.9s
[CV 4/5] END .....C=0.1, gamma=1, kernel=linear;, score=0.575 total time=  12.0s
[CV 5/5] END .....C=0.1, gamma=1, kernel=linear;, score=0.594 total time=  12.1s
[CV 1/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.389 total time=  35.4s
[CV 2/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.389 total time=  35.4s
[CV 3/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.389 total time=  37.2s
[CV 4/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.389 total time=  36.6s
[CV 5/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.389 total time=  34.8s
[CV 1/5] END ...C=0.1, gamma=0.1, kernel=linear;, score=0.593 total time=  13.3s
[CV 2/5] END ...C=0.1, gamma=0.1, kernel=linear

In [13]:
svm_classifier_1 = svm.SVC(kernel='rbf', C=1, gamma = 0.1)

scores = cross_val_score(svm_classifier_1, hog_features, y_train, cv=5, verbose=True)
scores

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:  1.7min finished


array([0.62064282, 0.59755545, 0.59846084, 0.60688406, 0.61231884])

In [17]:
svm_classifier_1.fit(hog_features, y_train)

SVC(C=1, gamma=0.1)

In [18]:
ypreds_val = svm_classifier_1.predict(hog_features_val)
print(classification_report(ypreds_val, y_val))

              precision    recall  f1-score   support

           1       0.53      0.64      0.58       116
           2       0.04      0.50      0.07         2
           3       0.00      0.00      0.00         3
           4       0.88      0.70      0.78       581
           5       0.40      0.56      0.47       154
           6       0.34      0.57      0.43        46
           7       0.68      0.49      0.57       326

    accuracy                           0.61      1228
   macro avg       0.41      0.49      0.41      1228
weighted avg       0.71      0.61      0.65      1228



SVM Model -3 

In [21]:
from sklearn.decomposition import PCA
import time

pca = PCA(n_components=100 , svd_solver='randomized' , whiten=True).fit(hog_features)

In [20]:
X_train_pca = pca.transform(hog_features)
X_val_pca = pca.transform(hog_features_val)

In [22]:
t0 = time.time()
svm_classifier_2 = svm.SVC(kernel='rbf', C=1, gamma = 0.1, class_weight='balanced')
svm_classifier_2.fit(X_train_pca, y_train)

ypreds_val = svm_classifier_2.predict(X_val_pca)
print(classification_report(ypreds_val, y_val))
print('Time Taken : ', (time.time()-t0))

              precision    recall  f1-score   support

           1       0.00      0.00      0.00         0
           2       0.00      0.00      0.00         0
           3       0.00      0.00      0.00         0
           4       1.00      0.38      0.55      1203
           5       0.00      0.00      0.00         0
           6       0.00      0.00      0.00         1
           7       0.06      0.58      0.11        24

    accuracy                           0.39      1228
   macro avg       0.15      0.14      0.09      1228
weighted avg       0.98      0.39      0.54      1228

Time Taken :  35.8096559047699


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


BEST MODEL

Best model is the base model with kernel = 'rbf' with HOG features. This is identified with accuracy as the metric on validation set and corss validations.


In [24]:
joblib.dump(svm_classifierCross, GOOGLE_DRIVE_PATH + '/Models/svm-hog.pkl')

['drive/My Drive/ColabNotebooks/ComputerVision/Course Work/CW_Folder_PG/Models/svm-hog.pkl']